# Imports and Path Settings

In [62]:
import pandas as pd
import numpy as np
import datetime
    
PWO_dataPath = "/Users/amaresh/Write-Off Data/May 2022 Write Off/Proposed Write Off - May 2022.xlsx" #update this with the link to the month's Proposed Write Off Doc



PWO_dropCols = ["Rep",
                "Bourgeois",
                "Shop Account Owner Name",
                'Email',
                      'Phone',
                      'Timezone',
                      'Shop Address',
                      'Shop Province/State',
                      'Shop City',
                      'Shop Postal/Zip Code',
                      'Plan Type',
                      'Current Exposure\n(USD)',
                      "Slow\nSales","Contact\nRequest",
                      "Contact\nRequest\nFollowup",
                      "Contact\nRequest\nFinal",
                      "Bad Act",
                      "Demand\nletter",
                      'Settle\n80%', #potentially needed for comments
                      "Settle\n50%",#potentially needed
                      "Settle\n20%",#potentially needed
                      "Pause\npayouts",
                      "Lock\nShop",
                      "SIFT/\nsoc media/\nalt store",
                      "TLOxp",
                      "UCC\nFiled",
                      "UCC\nSent",
                      'Team',
                      'followup date',
                      'Remittance\nDate and \nAmount',
                      'PAID']
                     

# IF followu up date column says SIF and write off column says just 'Yes' and not 'Yes - Settled'

# Converting Blanks to 'No Value'

In [63]:
df = pd.read_excel(PWO_dataPath)
df=df.fillna("No Value")
#print(df)
df.head()

,Shop ID,Shop Name,Rep,Bourgeois,Shop Country,Capital Currency,Financing ID,Financing Type,Shop Account Owner Name,Email,...,Lock\nShop,SIFT/\nsoc media/\nalt store,TLOxp,UCC\nFiled,UCC\nSent,followup date,write\noff,Remittance\nDate and \nAmount,PAID,Team
0,1,2,4.0,5,6,7,8,9,10,11,...,32,33,34,35,36,37,38,39,40,41
1,10077306944,TheGlamKitchen,No Value,Bourg,US,USD,175166,Merchant Cash Advance,Shauntay Parrish,theglamkitchen@gmail.com,...,No Value,No Value,No Value,No Value,No Value,2022-05-31 00:00:00,yes,No Value,No Value,Brass
2,10084360,Lock & Mane,No Value,Bourg,US,USD,202717,Merchant Cash Advance,Bendly Jean,bendly@ribbonequity.com,...,No Value,No Value,2022-05-26 00:00:00,No Value,No Value,2022-05-31 00:00:00,yes,No Value,No Value,Loonies
3,10133602389,SVGE New York,No Value,Bourg,US,USD,153566,Merchant Cash Advance,Trevin Smith,info@svge.nyc,...,No Value,No Value,2022-05-26 00:00:00,No Value,No Value,2022-05-31 00:00:00,yes,No Value,No Value,Loonies
4,10133602389,SVGE New York,No Value,Bourg,US,USD,175419,Merchant Cash Advance,Trevin Smith,info@svge.nyc,...,No Value,No Value,2022-05-26 00:00:00,No Value,No Value,2022-05-31 00:00:00,yes,No Value,No Value,Loonies


# Removing All the No's from the 'Write Off' Column

In [64]:
notNo_filter = df['write\noff'] != 'no' 
PWO_mainDF= df[notNo_filter] 

#PWO_mainDF.head()
#all data that is Yes or NAN in the Write-Off Column. Working DF because 'No Value ' is Yes

In [65]:
PWO_mainDF.shape[0]

981

# Verifying the 'No Value' Data in the 'Write Off' Column

In [66]:
noValue_filter = PWO_mainDF ['write\noff'] == 'No Value'
noValueData = PWO_mainDF[noValue_filter]
#noValueData #all data that is Yes or NAN in the Write-Off Column. Working DF because 'No Value ' is Yes

# This DF should usually be empty, check for anomalies. 

# Verify the 'Followup Date' Column for RIF, SIF, Legal, Bankrupt, Non-date Objects

In [67]:
PWO_followUpData = [] #Empty Array which will contain all the data from the above data that has a non-date time object

for index,row in PWO_mainDF.iterrows():
    if (isinstance(row['followup date'],datetime.datetime) != True) and (row['followup date'] != 'No Value'):
        PWO_followUpData.append(row)

In [68]:
followUpDataDF = pd.DataFrame(PWO_followUpData)
followUpDataDF = followUpDataDF[['Shop ID','Shop Name',"Shop Country", "Financing ID","Financing Type",'followup date','write\noff','Remittance\nDate and \nAmount','PAID','Team']] #Converting data from previous block to a Data Frame
followUpDataDF.head()

,Shop ID,Shop Name,Shop Country,Financing ID,Financing Type,followup date,write\noff,Remittance\nDate and \nAmount,PAID,Team
0,1,2,6,8,9,37,38,39,40,41
12,10507244,charitychristmascards,GB,202832,Merchant Cash Advance,SIF,yes settled - 50%,No Value,2022-12-30 00:00:00,Brass
27,11398728,garDspo,US,168418,Loan with Milestones,SIF,yes settled - 20%,No Value,2022-12-30 00:00:00,Brass
38,11861570,Bill McKim Photography -Jersey Shore whale wat...,US,116398,Merchant Cash Advance,SIF,yes settled - 50%,No Value,2022-12-30 00:00:00,Loonies
39,1197244477,HUCK+LIZ,US,164623,Merchant Cash Advance,SIF,yes settled - 50%,No Value,True,Loonies


In [69]:
#followUpDataDF.to_excel("/Users/amaresh/Data in Followup Date Column.xlsx")

# Dropping Columns in the 'Proposed Write Off' Doc - Dropped Columns listed above

In [70]:
PWO_finalWriteOffDF= PWO_mainDF.drop(PWO_dropCols,axis = 1)
PWO_finalWriteOffDF.head() #Final Dropped Columns DF before heading to the next step of the Write Off Process

,Shop ID,Shop Name,Shop Country,Capital Currency,Financing ID,Financing Type,Current Exposure\n(Capital Currency),Total Exposure\n(USD),write\noff
0,1,2,6,7,8,9,20.00,21.00,38
1,10077306944,TheGlamKitchen,US,USD,175166,Merchant Cash Advance,4369.06,4369.06,yes
2,10084360,Lock & Mane,US,USD,202717,Merchant Cash Advance,614.37,614.37,yes
3,10133602389,SVGE New York,US,USD,153566,Merchant Cash Advance,174.45,852.45,yes
4,10133602389,SVGE New York,US,USD,175419,Merchant Cash Advance,678.00,852.45,yes


In [71]:
finalWriteOffSheet= pd.DataFrame(PWO_finalWriteOffDF)
finalWriteOffSheet.head()


,Shop ID,Shop Name,Shop Country,Capital Currency,Financing ID,Financing Type,Current Exposure\n(Capital Currency),Total Exposure\n(USD),write\noff
0,1,2,6,7,8,9,20.00,21.00,38
1,10077306944,TheGlamKitchen,US,USD,175166,Merchant Cash Advance,4369.06,4369.06,yes
2,10084360,Lock & Mane,US,USD,202717,Merchant Cash Advance,614.37,614.37,yes
3,10133602389,SVGE New York,US,USD,153566,Merchant Cash Advance,174.45,852.45,yes
4,10133602389,SVGE New York,US,USD,175419,Merchant Cash Advance,678.00,852.45,yes


# Convert Final Recovery Write Off Sheet to Excel

In [72]:
finalWriteOffSheet.shape[0]

981

In [73]:
with pd.ExcelWriter("/Users/amaresh/Final Multi-Write Off Sheet.xlsx") as writer:
    finalWriteOffSheet.to_excel(writer,sheet_name="Final Write Off Sheet", index = False)#Sheet 1
    noValueData.to_excel(writer, sheet_name="Write Off - NA Data",index = False)
    followUpDataDF.to_excel(writer, sheet_name = 'Followup Date - non-date info',index = False)
    
    